<a href="https://colab.research.google.com/github/slin35/RobotProducer/blob/main/robotproducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import pandas as pd
import json
import numpy as np

In [215]:
# import data
data1 = pd.read_csv('/tmdb_5000_movies.csv')
data2 = pd.read_csv('/tmdb_5000_credits.csv')

data1 = data1[['genres', 'title', 'overview', 'id']]
data2 = data2[['movie_id', 'title', 'cast', 'crew']]

data = pd.merge(data1, data2, left_on='id', right_on='movie_id', how='inner')
data = data[['title_x', 'overview', 'genres', 'cast', 'crew']]
data.rename({'title_x': 'title'}, axis=1, inplace=True)



In [216]:
# cleanup genres
data['genres'] = [json.loads(i) if i != [] else [] for i in data['genres']]
data['genres'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['genres']]

# cleanup cast
data['cast'] = [json.loads(i) if i != [] else [] for i in data['cast']]
data['cast'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['cast']]

# cleanup crew
data['crew'] = [json.loads(i) if i != [] else [] for i in data['crew']]
data['crew'] = [[j['name']  for j in i if 'job' in j and j['job'] == 'Director'] for i in data['crew']]
data.tail()

,title,overview,genres,cast,crew
4798,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]","[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TV Movie]","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]
4802,My Date with Drew,Ever since the second grade when he first saw ...,[Documentary],"[Drew Barrymore, Brian Herzlinger, Corey Feldm...","[Brian Herzlinger, Jon Gunn, Brett Winn]"
